In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd
import urllib
import json
from tqdm import tqdm
import re

В ячейке ниже можно получить файл с яндекс диска

Настоятельно рекомендую пользоваться таким форматом, чтобы впоследствии любой код можно было запускать без предварительного поиска и локального скачивания файлов. 

In [2]:
folder_url = 'https://disk.yandex.ru/d/a7GgsLD0wo9E8g' #ссылка на папку "актуальные данные"
file_url = 'RUSSIAN_SOCIOLOGISTS_11_01_2023.xlsx' #тут должно быть имя файла
url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download' + '?public_key=' + urllib.parse.quote(folder_url) + '&path=/' + urllib.parse.quote(file_url)

r = requests.get(url) 
h = json.loads(r.text)['href']
df = pd.read_excel(h)

In [3]:
affiliations = []
for i in df.columns:
    if 'affiliation' in i:
        affiliations.append(i) #это список колонок в изначальном df, содержащих информацию об аффилиациях
        
af = pd.DataFrame() #создаю датасет сугубо из колонок с информацией по аффилиациям
for i in affiliations:
        af[i] = df[i]

['author_1_affiliation_id',
 'author_1_ru affiliation',
 'author_1_eng affiliation',
 'author_2_affiliation_id',
 'author_2_ru affiliation',
 'author_2_eng affiliation',
 'author_3_affiliation_id',
 'author_3_ru affiliation',
 'author_3_eng affiliation',
 'author_4_affiliation_id',
 'author_4_ru affiliation',
 'author_4_eng affiliation',
 'author_5_affiliation_id',
 'author_5_ru affiliation',
 'author_5_eng affiliation',
 'author_6_affiliation_id',
 'author_6_ru affiliation',
 'author_6_eng affiliation',
 'author_7_affiliation_id',
 'author_7_ru affiliation',
 'author_7_eng affiliation',
 'author_8_affiliation_id',
 'author_8_ru affiliation',
 'author_8_eng affiliation']

- Чистим id аффилиаций от всех ненужных знаков
- При помощи регулярных выражений оставляю только числовые значения 
- получившиеся после регулярного выражения пустые поля заполняю 'none'

In [5]:
ids = [a for a in affiliations if 'id' in a]
for a in ids:
        for m in tqdm(range(len(af[a]))):
            value = af[a][m].split(',')
            
            new_value = []
            for v in value:
                try:
                    v = re.sub("[^0-9]", "", v)
                    new_value.append(v)
                except:
                    v = v.strip("'").strip("[").strip("]")
                    new_value.append(v)
            
            for i in range(len(new_value)):
                if new_value[i] == '':
                    new_value[i] = 'none'
            
            af[a][m] = new_value

100%|██████████| 75232/75232 [01:15<00:00, 990.39it/s]


Теперь читаю сами аффилиации
- проверяю, есть ли внутри списка запятая между элементами, чтобы определить длину списка
- убираю специальные символы

In [7]:
af_train = af.copy() 
### создала отдельный df на всякий случай, потому что с диска грузится долго 
### (но не дольше чем с локального устройства :)  и обновлять его было бы неудобно

In [11]:
affs = [a for a in affiliations if 'id' not in a]
for a in affs:
        for m in tqdm(range(len(af_train[a]))):
            if "', '" in af_train[a][m]:
                value = af_train[a][m].split("', '")
                
            else:
                value = [af_train[a][m]]
                
            for v in range(len(value)):
                    value[v] = value[v].strip("[").strip("'").strip("]").strip("'")
            
            af_train[a][m] = value

100%|██████████| 75232/75232 [01:17<00:00, 965.35it/s]


In [74]:
af_train.head() 
###на этом шаге получается датасет, с которым уже можно работать, ибо он содержит все
###данные в нужных нам форматах

,author_1_affiliation_id,author_1_ru affiliation,author_1_eng affiliation,author_2_affiliation_id,author_2_ru affiliation,author_2_eng affiliation,author_3_affiliation_id,author_3_ru affiliation,author_3_eng affiliation,author_4_affiliation_id,...,author_5_eng affiliation,author_6_affiliation_id,author_6_ru affiliation,author_6_eng affiliation,author_7_affiliation_id,author_7_ru affiliation,author_7_eng affiliation,author_8_affiliation_id,author_8_ru affiliation,author_8_eng affiliation
0,"[7113, 7113]",[Белгородский государственный технологический ...,[Belgorod State Technological University named...,"[364, none]",[Новомосковский институт РХТУ им. Д.И. Менделе...,[none],"[none, none]",[none],[Novomoskovsk Institute RCTU named after D.I. ...,"[none, none]",...,[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none]
1,"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",...,[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none]
2,"[340, none]",[Белгородский государственный университет],[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",...,[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none]
3,"[210, none]",[Кубанский государственный университет],[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",...,[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none]
4,"[2541, none]",[МГУ им. М.В. Ломоносова?],[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",...,[none],"[none, none]",[none],[none],"[none, none]",[none],[none],"[none, none]",[none],[none]


In [ ]:
rus = [a for a in affiliations if 'ru' in a]
rus_af = []
for i in rus:
    rus_af.append(af_train[i].to_list())
rus_af = sum(rus_af, [])

In [ ]:
rus_af = sum(rus_af,[])

In [ ]:
a = list(set(rus_af))

In [ ]:
pd.Series(a).to_excel('all_affs_ru.xlsx')

In [15]:
a = pd.read_excel('all_affs_ru.xlsx')[0].to_list()

## Работа с ID аффилиаций

In [27]:
ru_to_id = {}
for i in a:
    ru_to_id[i] = []

Сначала создаю словарь, где ключи это аффилиации на русском, а их значения - списки, в которые будут добавляться ID 

In [28]:
for num in range(1,9):
    for i in tqdm(range(len(af_train))):
        for l in range(len(af_train['author_{}_ru affiliation'.format(num)][i])):
            try:
                ru_to_id[af_train['author_{}_ru affiliation'.format(num)][i][l]].append(af_train['author_{}_affiliation_id'.format(num)][i][l])
            except:
                print(af_train['author_{}_ru affiliation'.format(num)][i][l], i, l)

 22%|██▏       | 16927/75232 [00:00<00:01, 42388.85it/s]

mailto:vinokurova77@mail.ru&amp;gt 11075 1


100%|██████████| 75232/75232 [00:01<00:00, 42383.20it/s]


In [30]:
af_train['author_3_ru affiliation'][11075] = af_train['author_3_ru affiliation'][11075][0]

In [31]:
for i in ru_to_id.keys():
    ru_to_id[i] = list(set(ru_to_id[i]))

In [35]:
for i in ru_to_id.keys():
    if len(ru_to_id[i])>1:
        try:
            ru_to_id[i].remove('none')
        except:
            print([i, ru_to_id[i]])
            

'''
здесь ловим загадочные аффилиации, которые имели два разных ID
учитывая, что их таких две, полагаю, что проблема в данных, а не в коде
'''

['ГАОУ ВО ЛО «Ленинградский государственный университет имени А.С. Пушкина»', ['2149', '190']]
['БГУ', ['4706', '563']]


In [37]:
'''
здесь данные о аффилиациях, не имеющих ID вообще
планирую проверить, не имеют ли они случайно общих переводов / ID английского перевода
пока что получается, что 5771 аффилиаций из 18 000 не имеют ID
'''

counter = 0
for i in ru_to_id.keys():
    if ru_to_id[i] == ['none']:
        counter +=1
        #print([i, ru_to_id[i]])
print(counter)

5771


In [46]:
id_to_ru = {}

for i in tqdm(ru_to_id.keys()):
    for l in ru_to_id.keys():
        if ru_to_id[i]!=['none'] and ru_to_id[l]!=['none']:
            for m in ru_to_id[i]:
                if m in ru_to_id[l]:
                #if ru_to_id[i] in id_to_ru.keys():
                    try:
                        id_to_ru[m] += [i,l]
                    except:
                        id_to_ru[m] = [i,l]

100%|██████████| 18004/18004 [01:08<00:00, 262.66it/s]


In [49]:
for i in id_to_ru.keys():
    id_to_ru[i] = list(set(id_to_ru[i]))

In [65]:
lens = []
for i in id_to_ru.keys():
    if len(id_to_ru[i])>1:
        print(i, id_to_ru[i], '\n')
        print(len(id_to_ru[i]), '\n')
        

7082 ['Уральский институт управления РАНХиГС при Президенте РФ', 'Уральский институт-филиал Российской академии народного хозяйства и государственной службы при Президенте Российской Федерации (г. Екатеринбург)', 'Российская академия народного хозяйства и государственной службы при Президенте Российской Федерации- РАНХиГС', 'Алтайский филиал ФГБОУ ВО «Российская академия народного хозяйства и государственной службы при Президенте Российской Федерации', 'Международный институт госслужбы и управления Российской академии народного хозяйства и государственной службы при Президенте РФ', 'Владимирский филиал Российской академии народного хозяйства и государственной службы при Президенте Российской Федерации', 'ЮРИФ РАНХиГС', 'Российская академия народного хозяйства и государственной службы при Президенте Российской Федерации, Орловский филиал', 'Южно-Российский институт управления Российской академии народного хозяйства и государственного управления при Президенте Российской Федерации', 'Ура

4 

4898 ['Аналитический центр при Правительстве Российской Федерации', 'Аналитический центр при Правительстве РФ'] 

2 

555 ['Якутская государственная сельскохозяйственная академия', 'Арктический ГАТУ', 'ФГБОУ ВО Арктический ГАТУ'] 

3 

7141 ['Ессентукский институт управления бизнеса и права', 'частное образовательное учреждение высшего образования «Ессентукский институт управления, бизнеса и права»', 'Частное образовательное учреждение высшего образования Ессентукский институт управления, бизнеса и права', 'Частное образовательное учреждение высшего образования «Ессентукский институт управления, бизнеса и права»', 'ЧОУ ВО «Ессентукский институт управления бизнеса и права»', 'частное образовательное учреждение высшего образования «Ессентукский институт управления бизнеса и права»', 'Ессентукский институт управления, бизнеса и права', 'Частное образовательное учреждение высшего образования «Ессентукский институт управления бизнеса и права»'] 

8 

5350 ['Российская академия наук (Мос

In [91]:
lens = []
for i in id_to_ru.keys():
    lens.append(len(id_to_ru[i]))

In [93]:
itr = pd.DataFrame()
itr['id'] = id_to_ru.keys()
itr['vals'] = id_to_ru.values()
itr['len'] = lens

In [94]:
itr.head(10)

,id,vals,len
0,7082,[Уральский институт управления РАНХиГС при Пре...,718
1,14383,[Дальневосточный институт повышения квалификац...,1
2,1488,"[МГТУ Стакнин, МГТУ «СТАНКИН», ГТУ «Станкин», ...",27
3,15346,"[ОАО «РЖД», ОАО РЖД, ОАО Российские железные д...",4
4,741,[Кабардино Балкарский государственный универси...,29
5,3729,"[Сибирский юридический институт МВД России, Кр...",8
6,3298,[Липецкий государственный педагогический униве...,17
7,2268,[АО «Крибрум»],1
8,4007,"[ХТИ филиал СФУ, министерство культуры Красноя...",51
9,15628,[Поволжский православный институт],1


## Работа с английским переводом аффилиаций

In [52]:
ru_to_eng = {}
for i in a:
    ru_to_eng[i] = []

In [53]:
for num in range(1,9):
    for i in range(len(af_train)):
        for l in range(len(af_train['author_{}_ru affiliation'.format(num)][i])):
            try:
                ru_to_eng[af_train['author_{}_ru affiliation'.format(num)][i][l]].append(
                   af_train['author_{}_eng affiliation'.format(num)][i][l])
            except:
                print('По-русски:', af_train['author_{}_ru affiliation'.format(num)][i], '\n', 
                      'По-английски:', af_train['author_{}_eng affiliation'.format(num)][i], '\n')

По-русски: ['Южно российский филиал Учреждения Российской академии наук Института социологии РАН (ЮРФИС РАН)', 'Южный федеральный университет'] 
 По-английски: ['Southern Federal University'] 

По-русски: ['Институт социологии РАН', 'Новосибирский государственный университет'] 
 По-английски: ['none'] 

По-русски: ['Южнороссийский филиал Учреждения Российской академии наук Института социологии РА Н (ЮРФИС РА Н )', 'Южный федеральный университет'] 
 По-английски: ['none'] 

По-русски: ['Институт социологии РАН', 'Всероссийский государственный институт кинематографии им. С.А. Герасимова'] 
 По-английски: ['Institute of Sociology of the Russian Academy of Sciences'] 

По-русски: ['РГГУ', 'Институт социологии РАН'] 
 По-английски: ['none'] 

По-русски: ['Институт социологии РАН; Российско-Балтийский центр Института социологии', 'Институт экономики РАН;'] 
 По-английски: ['none'] 

По-русски: ['Нижегородский государственный университет им. Н.И. Лобачевского', 'Нижегородский институт менеджм


По-русски: ['Национальный исследовательский университет «Высшая школа экономики»', 'Центр исследований социальной организации фирмы (ЦИСОФ)'] 
 По-английски: ['none'] 

По-русски: ['Государственный университет – Высшая школа экономики', 'Институт социологии РАН'] 
 По-английски: ['none'] 

По-русски: ['Пермский государственный институт искусства и культуры', 'НИУ ВШЭ'] 
 По-английски: ['none'] 

По-русски: ['Российская академия архитектуры и строительных наук', 'Вологодский государственный университет'] 
 По-английски: ['Vologda State University'] 

По-русски: ['ФГАОУ ВПО «Северо-Кавказскогий федеральный университет, г. Ставрополь', 'Педагогический институт'] 
 По-английски: ['none'] 

По-русски: ['Дальневосточного института управления – филиала ФГБОУ ВПО «Российская академия народного хозяйства и государственной службы при Президенте РФ»', 'Дальневосточный институт повышения квалификации Федеральной службы РФ по контролю за оборотом наркотиков (г. Хабаровск)'] 
 По-английски: ['none'

По-русски: ['Московский гуманитарный университет', 'Центр патологии речи и нейрореабилитации'] 
 По-английски: ['none'] 

По-русски: ['Государственный университет - Высшая школа экономики', 'Институт социологии РАН'] 
 По-английски: ['none'] 

По-русски: ['Национальный исследовательский университет «Высшая школа экономики» (Санкт-Петербург)', 'НИЦ «Регион» (Ульяновский государственный университет)'] 
 По-английски: ['none'] 

По-русски: ['Национальный исследовательский университет “Высшая школа экономики”', 'Институт фонда “Общественное мнение”'] 
 По-английски: ['none'] 

По-русски: ['государственное автономное учреждение «Институт переподготовки и повышения квалификации руководящих кадров и специалистов системы социальной защиты населения г. Москвы»', 'ЦСИИП «Десница Мира»'] 
 По-английски: ['none'] 

По-русски: ['Национальный исследовательский университет «Высшая школа экономики»', 'ФГБУ ВПО «Саратовский государственный технический университет им. Гагарина Ю.А.»'] 
 По-английски: ['

По-русски: ['РГГУ', 'Институт культурологии'] 
 По-английски: ['none'] 

По-русски: ['Самарский государственный университет', 'Федеральный научно-исследовательский социологический центр РАН'] 
 По-английски: ['none'] 

По-русски: ['Федеральное Государственное бюджетное образовательное учреждение высшего профессионального образования «Северо-Осетинский государственный университет им. К.Л. Хетагурова»', 'Цхинвальский многопрофильный колледж'] 
 По-английски: ['none'] 

По-русски: ['Российский государственный гуманитарный университет', 'Журнал Социологические исследования'] 
 По-английски: ['Russian State University for the Humanities'] 

По-русски: ['Национальная Академия наук Республики Казахстан', 'Институт философии, политологии и религиоведения Министерства образования и науки Республики Казахстан'] 
 По-английски: ['Institute of philosophy, political research, and religion studies of the Republic of Kazakhstan'] 

По-русски: ['РАНХиГС при Президенте РФ (Воронежский филиал)', 'Инстит

По-русски: ['Институт социального анализа и прогнозирования РАНХиГС при Президенте РФ', 'Институт социологии РАН'] 
 По-английски: ['The Russian Presidential Academy of National Economy and Public Administration (RANEPA)'] 

По-русски: ['Институт социологии РАН', 'Государственный академический университет гуманитарных наук'] 
 По-английски: ['State academic university for humanity'] 

По-русски: ['Санкт-Петербургский государственный университет', 'Российско-Китайский Центр сравнительных социальных, экономических и политических исследований факультета социологии'] 
 По-английски: ['none'] 

По-русски: ['Российская академия народный хозяйства и государственной службы при Президенте Российской Федерации', 'Поволжский институт управления имени П.А. Столыпина'] 
 По-английски: ['none'] 

По-русски: ['Саратовский государственный технический университет', 'Центр социальной политики и гендерных исследований'] 
 По-английски: ['none'] 

По-русски: ['Институт социологии РАН', 'Государственный ак

...

По-русски: ['Институт гуманитарных историко-теоретических исследований имени А.В. Полетаева (ИГИТИ)', 'НИУ ВШЭ'] 
 По-английски: ['none'] 

По-русски: ['ИППК ЮФУ', 'ЮРФИС РАН'] 
 По-английски: ['none'] 

По-русски: ['Волгоградский институт управления - филиал Российской академии народного хозяйства и государственной службы при Президенте РФ', 'Волгоградский государственный университет'] 
 По-английски: ['none'] 

По-русски: ['Российская академия народного хозяйства и государственной службы при Президенте РФ', 'филиал'] 
 По-английски: ['none'] 

По-русски: ['Уральский федеральный университет', 'Институт государственного и муниципального управления'] 
 По-английски: ['none'] 

По-русски: ['Институт социологии РАН', 'Государственный академический университет гуманитарных наук'] 
 По-английски: ['none'] 

По-русски: ['Санкт-Петербургский государственный университет', 'Российско-Китайский Центр сравнительных социальных, экономических и политических исследований факультета социологии Санкт-

По-русски: ['Научно-исследовательский институт организации здравоохранения и медицинского менеджмента', 'Департамент здравоохранения города Москвы'] 
 По-английски: ['State-financed Institution of Moscow city "Scientific Research Institute of Healthcare Organization and Medical Management of Moscow Department of Healthcare"'] 

По-русски: ['Федеральный научно-исследовательский социологический центр РАН', 'Государственный академический университет гуманитарных наук'] 
 По-английски: ['none'] 

По-русски: ['Московский гуманитарный университет', 'Федеральный научно-исследовательский социологический центр РАН'] 
 По-английски: ['Federal Center of Theoretical and Applied Sociology of the Russian Academy of Sciences'] 

По-русски: ['Российский университет дружбы народов', 'Институт социологии ФНИСЦ РАН'] 
 По-английски: ['none'] 

По-русски: ['Университет Калгари', 'Гонконгский университет'] 
 По-английски: ['University of Calgary'] 

По-русски: ['Южнороссийский филиал Федерального научно-ис

По-русски: ['Московская финансово-юридическая академия', 'Московский государственный институт электроники и математики (МИЭМ)', 'Национальный транспортный морской университет Вьетнама'] 
 По-английски: ['none'] 

По-русски: ['Московская финансово-юридическая академия', 'Московский государственный институт электроники и математики (МИЭМ)', 'Национальный транспортный морской университет Вьетнама'] 
 По-английски: ['none'] 

По-русски: ['ГОУ ВПО «Нижегородская Государственная медицинская академия министерства здравоохранения и социального развития Российской Федерации»', 'ФГУЗ МСЧ ГУВД по Нижегородской области'] 
 По-английски: ['none'] 

По-русски: ['Законодательное собрание Тверской области по социальной политике', 'НОУ ВПО ИГУПИТ'] 
 По-английски: ['The Legislative Assembly of Tver region for social policy'] 

По-русски: ['Государственный университет - Высшая школа экономики', 'ФОМ'] 
 По-английски: ['none'] 

По-русски: ['Фонд «Общественное мнение»', 'ГУ-ВШЭ'] 
 По-английски: ['none']

По-русски: ['ООО &amp;quot', 'Реванш&amp;quot', '(г. Саратов)'] 
 По-английски: ['none'] 

По-русски: ['ООО &amp;quot', 'Реванш&amp;quot', '(г. Саратов)'] 
 По-английски: ['none'] 

По-русски: ['Южный федеральный университет', 'Южнороссийский филиал Института Социологии РАН', 'Российская академия естественных наук'] 
 По-английски: ['Southern Federal University'] 

По-русски: ['Южный федеральный университет', 'Южнороссийский филиал Института Социологии РАН', 'Российская академия естественных наук'] 
 По-английски: ['Southern Federal University'] 

По-русски: ['КНИТУ', 'KNRTU'] 
 По-английски: ['none'] 

По-русски: ['Башкирский государственный педагогический университет им. М. Акмуллы', 'Уфимский государственный университет экономики и сервиса'] 
 По-английски: ['none'] 

По-русски: ['Южнороссийский филиал Института социологии РАН', 'ИППК Южного федерального университета'] 
 По-английски: ['none'] 

По-русски: ['Институт социологии РАН', 'Государственный академический университет гумани

По-русски: ['Социологический институт РАН (Санкт-Петербург)', 'Социологический институт РАН'] 
 По-английски: ['Sociological Institute, Russian Academy of Sciences (St. Petersburg)'] 

По-русски: ['Саньяский университет', 'ДВФУ'] 
 По-английски: ['none'] 

По-русски: ['Чурапчинский государственный институт физической культуры и спорта', 'Северо-Восточный федеральный университет'] 
 По-английски: ['none'] 

По-русски: ['Институт философии и социально-политических наук Южного федерального университета', 'Южно-Российский институт управления - филиал РАНХиГС при Президенте РФ'] 
 По-английски: ['South Russian Institute of Management - brunch of the Russian Presidential Academy of National Economy and Public Administration'] 

По-русски: ['Южный федеральный университет', 'Южно-Российский институт управления - филиал Российской академии народного хозяйства и государственной службы при Президенте РФ'] 
 По-английски: ['South-Russian Institute of Management - branch of the Russian Presidential

In [54]:
for i in ru_to_eng.keys():
    ru_to_eng[i] = list(set(ru_to_eng[i]))

In [69]:
for i in ru_to_eng.keys():
    if ru_to_eng[i] == []:
        ru_to_eng[i] = ['none']

In [82]:
ru_to_eng

{'АНО «Социальная валидация»': ['none'],
 'Государственный института искусствознания (Москва)': ['none'],
 'Московский областной филиал Российской академии народного хозяйства и государственной службы при Президенте Российской Федерации': ['none'],
 'Социально-гуманитарный парк': ['none'],
 'Дальневосточный институт повышения квалификации ФСКН России': ['none'],
 'Институт конструкторско-технологической информации РАН Московский государственный технологический университет «Станкин»': ['none'],
 'ОАО РЖД': ['none'],
 'Кабардино-Балкарский государственный университет имени Х.М. Бербекова': ['Berbekov Kabardino-Balkar State University',
  'Berbekov Kabardino-Balkarian State University',
  'Kabardino-Balkar state University named after H. M. berbe-kov',
  'none',
  'Kabardino-Balkar state university n.a. H.M. Berbekov'],
 'Сибирский юридический институт МВД РФ': ['Federal State Public Educational Institute of Higher Education "Siberian Law Institute of the Ministry of Internal Affairs of t

In [70]:
af_df = pd.DataFrame()
af_df['ru_aff'] = list(ru_to_id.keys())
af_df['id'] = [ru_to_id[i] for i in ru_to_id.keys()]
af_df['eng_aff'] = [ru_to_eng[i] for i in ru_to_eng.keys()]

In [71]:
af_df['eng_aff'].value_counts()

[none]                                                                                                                                                                                                                                                                                 8365
[Russian Presidential Academy of National Economy and Public Administration]                                                                                                                                                                                                             25
[National Research University Higher School of Economics]                                                                                                                                                                                                                                21
[Lomonosov Moscow State University]                                                                                                                 

In [96]:
af_df.to_excel('af_df.xlsx')

# Теперь это все надо свести вместе, и мы над этим работаем...

In [97]:
pip install fastparquet

  Using cached pandas-1.3.5-cp37-cp37m-macosx_10_9_x86_64.whl (11.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 0.25.1
    Uninstalling pandas-0.25.1:
      Successfully uninstalled pandas-0.25.1
Note: you may need to restart the kernel to use updated packages.


In [99]:
af_df.to_parquet('af_df.parquet')

In [100]:
pd.read_parquet('af_df.parquet')

,ru_aff,id,eng_aff
0,АНО «Социальная валидация»,[none],[none]
1,Государственный института искусствознания (Мос...,[none],[none]
2,Московский областной филиал Российской академи...,[7082],[none]
3,Социально-гуманитарный парк,[none],[none]
4,Дальневосточный институт повышения квалификаци...,[14383],[none]
...,...,...,...
17999,Институт экономической политики им. Е.Т. Гайдара,[574],[Gaidar Institute of Economic Policy]
18000,Красноярский ГАУ,[6172],[none]
18001,Институт лингвистических исследований РАН,[6262],[Institute for Linguistic Studies]
18002,ФБОУ ВПО «Пермский национальный исследовательс...,[848],[FSBEI HPE «!Perm National Research Polytechni...
